In [46]:
import torch, io, json

In [47]:
print(torch.__version__)

1.7.0


In [48]:
with open('generated_features/val/val_expression.pt', 'rb') as f:
  buffer = io.BytesIO(f.read())
expression_results=torch.load(buffer,map_location=torch.device('cpu'));

In [49]:
with open('generated_features/val/val_object_detection.pt', 'rb') as f:
  buffer = io.BytesIO(f.read())
object_results=torch.load(buffer,map_location=torch.device('cpu'));

In [58]:
with open("generated_features/val/val_places365.json", "r") as json_file: 
    places_result=json.load(json_file)

**Get classes**

In [55]:
# Get expression classes
with open("class_files/expression_classes.txt", "r") as f:
    expression_classes = [s.strip() for s in f.readlines()]

In [56]:
# Get coco classes
with open("class_files/coco_classes.txt", "r") as f:
    coco_classes = [s.strip() for s in f.readlines()]

In [57]:
# Get places365 classes
file_name = 'class_files/coco_classes.txt'                                                                                                                                       
places365_classes = list()                                                                                         
with open(file_name) as class_file:                                                                      
    for line in class_file:                                                                              
        places365_classes.append(line.strip().split(' ')[0][3:])  

**Test output**

In [61]:
# Places
places_result['2017_10735550']

{'prob': [0.5115284323692322,
  0.1368655413389206,
  0.07560905069112778,
  0.05295965448021889,
  0.03759666159749031],
 'class': ['museum/indoor',
  'burial_chamber',
  'cemetery',
  'archaelogical_excavation',
  'kindergarden_classroom']}

In [84]:
# Coco
detections=object_results[15]['output']['labels']
for d in detections[:10]:
    print(coco_classes[d])

clock
umbrella
surfboard
frisbee
bowl
keyboard
person
cup
dining table
tv


In [110]:
# Expressions
face_expression=expression_results[0]['2017_90320159'][0]['0']['classes']
for face in face_expression:
    print(expression_classes[int(face)])

neutral
anger
sadness
